# Demo - inserting lines of clinical data into xnat for archival

### import necessary components

In [ ]:
#ORAW modifications and integrations with XNAT by Leonard Wee during Oct -> Dec 2021
from __future__ import print_function
from datetime import date, datetime
import os, glob

import xnat #needed for reading and writing to/from XNAT

### define needed operations

In [ ]:
def upload_file(session, project, subject, experiment, assessment, resource, convfile):
    xnat_project = session.projects[project]
    xnat_subject = xnat_project.subjects[subject]
    xnat_experiment = xnat_project.experiments[experiment]
    xnat_resource = session.classes.ResourceCatalog(parent=xnat_experiment, label=resource)
    #xnat_assessment = session.classes.QcAssessmentData(parent=xnat_experiment, label=assessment) #not used
    #xnat_resource = session.classes.ResourceCatalog(parent=xnat_assessment, label=resource) #resource under experiment instead
    xnat_resource.upload(convfile, os.path.basename(convfile)) # upload
#    for file_ in data:
#        resource.upload(file_, os.path.basename(file_))
#    pass


def cleanup_temp_folder(dirt):
    if os.path.exists(dirt):
        shutil.rmtree(dirt)
        os.makedirs(dirt)
    if not os.path.exists(dirt):
        os.makedirs(dirt)

### configure user-dependent settings

In [ ]:
#------------------------- USER SETTINGS -------------------------------

# set up XNAT login credentials here
# .......................
#xnatUrl = 'https://xnat.bmia.nl'
#xnatUser = 'leonardwee'
xnatUrl = 'http://xnat-nginx:80'  #change me! e.g. 'http://localhost:8081/'
xnatUser = 'admin'  #change me!
xnatPass = 'admin'  #change me!
xnatProject = 'flyover-05'  #change me

### retrieve list of patients and dicom experiments from XNAT project

In [ ]:
with xnat.connect(xnatUrl, user=xnatUser, password=xnatPass) as session:
    myProject= session.projects[xnatProject]
    mySubjectsList = myProject.subjects.values()
    for s in mySubjectsList:
        mySubjectID = s.label
        mySubject = myProject.subjects[mySubjectID]
        myExperimentsList = mySubject.experiments.values()
        for e in myExperimentsList:
            myExperimentID = e.label
            myExperiment = mySubject.experiments[myExperimentID]
            print(xnatProject + "\t" + mySubjectID + "\t" + myExperimentID)

### uploading structured clinical per subject

In [ ]:
with xnat.connect(xnatUrl, user=xnatUser, password=xnatPass) as session:
    try:
        upload_file(session, xnatProject, 'LUNG1-001', 'LUNG1-001', '0', 'lung1-001-clinical', '/home/jovyan/work/data/lung1-001-clinical.csv')
        upload_file(session, xnatProject, 'LUNG1-002', 'LUNG1-002', '0', 'lung1-002-clinical', '/home/jovyan/work/data/lung1-002-clinical.csv')
        upload_file(session, xnatProject, 'LUNG1-003', 'LUNG1-003', '0', 'lung1-003-clinical', '/home/jovyan/work/data/lung1-003-clinical.csv')
    except:
        print("--- error during demo for inserting clinical data! -----")

### extracting clinical elements and consolidating as single dataframe

In [7]:
ClinicalWorkingDir = './pre-process-CLINICAL'
ClinicalSetLabel = '-clinical'

#CAREFUL - empties the following before filling it up from XNAT
cleanup_temp_folder(ClinicalWorkingDir)

# -----------------------------------------------------------
with xnat.connect(xnatUrl, user=xnatUser, password=xnatPass) as session:
    myProject = session.projects[xnatProject]
    mySubjectsList = myProject.subjects.values()
    for s in mySubjectsList:
            mySubjectID = s.label
            mySubject = myProject.subjects[mySubjectID]
            myExperimentsList = mySubject.experiments.values()
            for e in myExperimentsList:
                myExperimentID = e.label
                myExperiment = mySubject.experiments[myExperimentID]
                myResourcesList = myExperiment.resources.values()
                for r in myResourcesList:
                    myResourceID = r.label
                    if myResourceID.endswith(ClinicalSetLabel):
                        myResource = myExperiment.resources[myResourceID]
                        for i in range(len(myResource.files)):
                            myFile = myResource.files[i].data['Name']
                            myResource.files[i].download(os.path.join(ClinicalWorkingDir,myFile))


[WARNING] Detected a redirect from http://xnat-nginx:80 to http://xnat-nginx/, using http://xnat-nginx/ from now on
100% of 179.0 B |##################################| 144.1 KiB/s Time:  0:00:00
100% of 188.0 B |##################################| 180.5 KiB/s Time:  0:00:00
100% of 178.0 B |##################################| 156.7 KiB/s Time:  0:00:00


In [8]:
import pandas as pd

all_files = glob.glob(ClinicalWorkingDir + "/*.csv")
li = []
for fname in all_files:
    try :
        df = pd.read_csv(fname, index_col=None, header=0)
        li.append(df)
    except :
        print("This did not have the expected data!")

frame = pd.concat(li, axis=0, ignore_index=True)

frame.to_csv('./test.csv', index=False) ###change this filename